In [ ]:
%pip install llama-index-embeddings-azure-openai
%pip install llama-index-llms-azure-openai

In [ ]:
%pip install llama-index


In [ ]:
%pip install llama-index chromadb --quiet
%pip install chromadb
%pip install sentence-transformers
%pip install pydantic==1.10.11

In [1]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

/Users/ogb/projects/cal/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import getpass
import os
# # MSFT
openai_api_key = getpass.getpass("Enter your OpenAI API Key: ")
os.environ["AZURE_OPENAI_API_KEY"] = openai_api_key


In [3]:
api_key = openai_api_key
azure_endpoint = "https://cursor-gpt-4.openai.azure.com"
api_version="2024-02-15-preview"


llm = AzureOpenAI(
    model="gpt-35-turbo",
    deployment_name="pjf-dpo-turbo-35",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

response = llm.complete("Paul Graham is ")
print(response)

INFO:httpx:HTTP Request: POST https://cursor-gpt-4.openai.azure.com/openai/deployments/pjf-dpo-turbo-35/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://cursor-gpt-4.openai.azure.com/openai/deployments/pjf-dpo-turbo-35/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
a computer scientist, entrepreneur, and venture capitalist. He is best known for co-founding the startup accelerator Y Combinator and for his work on the programming language Lisp. Graham has also written influential essays on startups, technology, and entrepreneurship.


In [9]:
# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="ada_gcal",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

In [11]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [5]:
# import pymongo
# from llama_index.vector_stores.azurecosmosmongo import (
#     AzureCosmosDBMongoDBVectorSearch,
# )
# from llama_index.core import VectorStoreIndex
# from llama_index.core import StorageContext
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

print("Document ID:", documents[0].doc_id)


Document ID: 7dad6038-ea0e-4ec1-b7e3-693be983f663


In [ ]:
%pip install llama-index-vector-stores-chroma
%pip install llama-index-embeddings-huggingface

In [6]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex

# global
# Settings.embed_model = OpenAIEmbedding()

# per-index
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

INFO:httpx:HTTP Request: POST https://cursor-gpt-4.openai.azure.com/openai/deployments/ada_gcal/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://cursor-gpt-4.openai.azure.com/openai/deployments/ada_gcal/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://cursor-gpt-4.openai.azure.com/openai/deployments/ada_gcal/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://cursor-gpt-4.openai.azure.com/openai/deployments/ada_gcal/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://cursor-gpt-4.openai.azure.com/openai/deployments/ada_gcal/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://cursor-gpt-4.openai.azure.com/openai/deployments/ada_gcal/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"


In [13]:
from IPython.display import Markdown, display
query_engine = index.as_query_engine()

response = query_engine.query("What did the author do growing up?")

print(response.get_formatted_sources())

print("response was:")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://cursor-gpt-4.openai.azure.com/openai/deployments/ada_gcal/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://cursor-gpt-4.openai.azure.com/openai/deployments/ada_gcal/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://cursor-gpt-4.openai.azure.com/openai/deployments/pjf-dpo-turbo-35/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://cursor-gpt-4.openai.azure.com/openai/deployments/pjf-dpo-turbo-35/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
> Source (Doc id: d7361350-0899-49ee-953a-7e910912fb70): What I Worked On

February 2021

Before college the two main things I worked on, outside of schoo...

> Source (Doc id: 1404bdfa-51b3-4917-b382-f70e275437e2): Now that I could write essays again, I wrote a bunch about topics I'd had stacked up. I kept writ...
response was:


<b>The author worked on writing and programming outside of school before college. They wrote short stories and tried writing programs on the IBM 1401 using an early version of Fortran. Later, with the advent of microcomputers, the author started programming more extensively, writing simple games, a program to predict rocket heights, and a word processor.</b>

In [45]:
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb

# create client and a new collection
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("quickstart")

# define embedding function
# embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

# load documents
# documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

# set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

# Query Data
query_engine = index.as_query_engine()

UniqueConstraintError: Collection quickstart already exists

In [46]:
response = query_engine.query("What did the author enjoy?")
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://cursor-gpt-4.openai.azure.com/openai/deployments/ada_gcal/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://cursor-gpt-4.openai.azure.com/openai/deployments/ada_gcal/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://cursor-gpt-4.openai.azure.com/openai/deployments/pjf-dpo-turbo-35/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://cursor-gpt-4.openai.azure.com/openai/deployments/pjf-dpo-turbo-35/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"


<b>The author enjoyed programming and writing, particularly during their early years before college. They found joy in writing short stories and programming on the IBM 1401, as well as later on microcomputers like the TRS-80.</b>

In [47]:
display(Markdown(f"<b>{response.get_formatted_sources()}</b>"))


<b>> Source (Doc id: 2f780940-5312-437b-aa84-6edbcbd2f7bc): Now that I could write essays again, I wrote a bunch about topics I'd had stacked up. I kept writ...

> Source (Doc id: c4ba9e98-3cb7-4496-b4bf-769850d4945c): What I Worked On

February 2021

Before college the two main things I worked on, outside of schoo...</b>